In [1]:
!git clone https://github.com/AleksL04/ML-Project.git
%cd ML-Project

Cloning into 'ML-Project'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 50 (delta 22), reused 42 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 22.94 KiB | 22.94 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/ML-Project


In [2]:
!git pull

Already up to date.


In [3]:
%%capture
!pip install --upgrade gensim
!gdown --folder 1rl_TJMHtcP-S0fTHNUpr9oi8Ndz2Y5z4 -O project_data
!gdown --folder 1RmC_We2lfsJontwrxlc4dzqEY_mNy-UW -O word2vec_model

In [4]:
ls -l

total 48
-rw-r--r-- 1 root root 32286 Dec 10 23:02 main_console.ipynb
drwxr-xr-x 2 root root  4096 Dec 10 23:02 model_code/
-rw-r--r-- 1 root root  2319 Dec 10 23:02 process_df.py
drwxr-xr-x 2 root root  4096 Dec 10 23:02 project_data/
drwxr-xr-x 2 root root  4096 Dec 10 23:03 word2vec_model/


In [5]:
import pandas as pd
import numpy as np

In [6]:
train_df = pd.read_csv('project_data/train.csv')
valid_df = pd.read_csv('project_data/valid.csv')
test_df = pd.read_csv('project_data/test.csv')

In [7]:
from process_df import process_df

X_train = process_df(train_df)
y_train = train_df['label'].values

X_valid = process_df(valid_df)
y_valid = valid_df['label'].values

X_test = process_df(test_df)
y_test = test_df['label'].values

In [ ]:
from process_df import process_df
from model_code.LSTM_code import trainLSTM
from model_code.CNN_code import trainCNN

In [ ]:
X_train_text, X_train_feat = process_df(train_df)
X_valid_text, X_valid_feat = process_df(valid_df)
X_test_text, X_test_feat = process_df(test_df)

# Training LSTM
LSTM_model = trainLSTM(
    [X_train_text, X_train_feat], y_train, 
    [X_valid_text, X_valid_feat], y_valid
)

train_preds = LSTM_model.predict(X_train)

y_train_flat = y_train.flatten() if hasattr(y_train, 'flatten') else np.array(y_train).flatten()
preds_flat = train_preds.flatten()

errors = np.abs(y_train_flat - preds_flat)

sample_weights = 1.0 + (errors * 10.0)

# Training CNN
CNN_model = trainCNN(
    [X_train_text, X_train_feat], y_train, 
    [X_valid_text, X_valid_feat], y_valid, 
    sample_weights
)

In [8]:
from model_code.LSTM_code import trainLSTM

LSTM_model = trainLSTM(X_train, y_train, X_valid, y_valid)

test_loss, test_acc = LSTM_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8743 - loss: 0.2532
Test Accuracy: 0.8861


671/671 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step


In [10]:
from model_code.CNN_code import trainCNN

CNN_model = trainCNN(X_train, y_train, X_valid, y_valid, sample_weights)

test_loss, test_acc = CNN_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8062 - loss: 0.4352
Test Accuracy: 0.8095


In [11]:
val_pred_LSTM = LSTM_model.predict(X_valid)
val_pred_CNN = CNN_model.predict(X_valid)

X_meta_train = np.column_stack((val_pred_LSTM, val_pred_CNN))

y_meta_train = y_valid

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


In [12]:
from model_code.Blender_code import trainBlender

Blender_model = trainBlender(X_meta_train, y_meta_train)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.4516 - loss: 0.6945 - learning_rate: 0.0010
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5441 - loss: 0.6941 - learning_rate: 0.0010
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4683 - loss: 0.6934 - learning_rate: 0.0010
Epoch 4/100


/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/callback_list.py:145: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4529 - loss: 0.6936 - learning_rate: 0.0010
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5161 - loss: 0.6931 - learning_rate: 0.0010
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5446 - loss: 0.6926 - learning_rate: 0.0010
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5846 - loss: 0.6921 - learning_rate: 0.0010
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7157 - loss: 0.6882 - learning_rate: 0.0010
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6359 - loss: 0.6844 - learning_rate: 0.0010
Epoch 10/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6117 - loss: 0.6770 - learning_rate: 0.0010
Epoch 11/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5614 - loss: 0.6700 - learning_rate: 0.0010
Epoch 12/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5501 - loss: 0.6633 - learning_rate: 0.0010
Epoch 13/100
23/23 ━━━━━━━━━

In [13]:
test_pred_LSTM = LSTM_model.predict(X_test)
test_pred_CNN = CNN_model.predict(X_test)

X_meta_test = np.column_stack((test_pred_LSTM, test_pred_CNN))

test_loss, test_acc = Blender_model.evaluate(X_meta_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8828 - loss: 0.3241
Test Accuracy: 0.8892
